In [10]:
def calculate_checksum(file_path):
    # Step 1: Read the input disk map from the file
    with open(file_path, 'r') as file:
        disk_map = file.read().strip()

    # Step 2: Parse the disk map into file lengths and free spaces
    layout = []
    current_file_id = 0  # Start with file ID 0
    i = 0
    while i < len(disk_map):
        length = int(disk_map[i])  # Get the block length
        if i % 2 == 0:  # Even indices represent file lengths
            layout.extend([current_file_id] * length)  # Repeat file ID by the length of the file
            current_file_id += 1  # Increment the file ID after processing a file
        else:  # Odd indices represent free spaces
            layout.extend(['.'] * length)  # Add free space
        i += 1

    # Step 3: Compact the disk by moving file blocks into the leftmost available spaces
    compacted = []
    for block in layout:
        if block != '.':  # If it's a file block, add it to compacted disk
            compacted.append(block)

    # Fill the remaining space with free blocks at the end
    compacted.extend(['.'] * layout.count('.'))

    # Step 4: Calculate the checksum by summing position * file ID for each file block
    checksum = 0
    position = 0
    for block in compacted:
        if block != '.':  # Skip free space blocks
            checksum += position * block  # Multiply position by file ID
            position += 1  # Increment position for each file block
        else:
            position += 1  # Skip positions for free spaces

    return checksum


# Example usage (use the correct path to your input file)
file_path = '/content/drive/MyDrive/Personal Project/Advent of Code/2024/Day_9/input_09_12_2024.txt'
result = calculate_checksum(file_path)
print("Filesystem Checksum:", result)


Filesystem Checksum: 8480081849953


In [8]:
import copy

def clean_up_disk(tupledisk):
    disk = copy.deepcopy(tupledisk)  # Prevent modifying original data
    compact_disk = [disk[0]]  # Start with the first file
    indexfree, indexlast = 1, len(disk) - 1

    # Make the compact disk by moving file blocks into the leftmost available spaces
    while indexfree <= indexlast:
        if disk[indexfree][1] == disk[indexlast][1]:
            # Same size blocks, just swap them
            compact_disk.extend([disk[indexlast], disk[indexfree + 1]])
            indexfree += 2
            indexlast -= 2
        elif disk[indexfree][1] > disk[indexlast][1]:
            # File is larger than free space, move the file and reduce its size
            compact_disk.append(disk[indexlast])
            disk[indexfree] = (disk[indexfree][0], disk[indexfree][1] - disk[indexlast][1])
            indexlast -= 2
        else:
            # Free space is larger than the file, move part of it
            compact_disk.append((disk[indexlast][0], disk[indexfree][1]))
            disk[indexlast] = (disk[indexlast][0], disk[indexlast][1] - disk[indexfree][1])
            compact_disk.append(disk[indexfree + 1])
            indexfree += 2

    return compact_disk


def check_sum(compact_disk):
    checksum, position = 0, 0

    # Calculate the checksum by summing position * file ID for each file block
    for block in compact_disk:
        file_id, length = block
        if file_id != "free":  # Only consider file blocks
            for _ in range(length):
                checksum += position * file_id
                position += 1
        else:  # Skip free space
            position += length

    return checksum


def parse_disk_map(disk_map):
    # Parse the disk map into a list of tuples: (file_id, length) or ("free", length)
    layout = []
    file_id = 0
    i = 0
    while i < len(disk_map):
        length = int(disk_map[i])
        if i % 2 == 0:  # Even indices represent file blocks
            layout.append((file_id, length))  # Add file block with current file ID
            file_id += 1  # Increment file ID for the next file
        else:  # Odd indices represent free space
            layout.append(("free", length))  # Add free space
        i += 1

    return layout


def main(file_path):
    # Main logic to read the disk map, compact it, and calculate checksum
    with open(file_path, 'r') as file:
        disk_map = file.read().strip()  # Read and clean the input data

    # Parse the disk map into a list of file IDs and free spaces
    layout = parse_disk_map(disk_map)

    # Compact the disk by removing gaps
    compacted_disk = clean_up_disk(layout)

    # Calculate and print the checksum of the compacted disk
    checksum = check_sum(compacted_disk)
    print("Filesystem Checksum:", checksum)


# Example usage (use the correct path to your input file)
file_path = '/content/drive/MyDrive/Personal Project/Advent of Code/2024/Day_9/input_09_12_2024.txt'
main(file_path)


Filesystem Checksum: 6471961544878


In [ ]:
import copy


def clean_up_disk(tupledisk):
    disk = copy.deepcopy(tupledisk)  # otherwise it changes thing for part 2
    compact_disk = [disk[0]]
    indexfree, indexlast = 1, len(disk) - 1

    # make the compact disk
    while indexfree <= indexlast:
        if disk[indexfree][1] == disk[indexlast][1]:
            # same size replace
            compact_disk.extend([disk[indexlast], disk[indexfree + 1]])
            indexfree += 2
            indexlast -= 2
        elif disk[indexfree][1] > disk[indexlast][1]:
            # put it here and change free size
            compact_disk.append(disk[indexlast])
            disk[indexfree] = (disk[indexfree][0], disk[indexfree][1] - disk[indexlast][1])
            indexlast -= 2
        else:
            # put partly here and keep in mind other part
            compact_disk.append((disk[indexlast][0], disk[indexfree][1]))
            disk[indexlast] = (disk[indexlast][0], disk[indexlast][1] - disk[indexfree][1])
            compact_disk.append(disk[indexfree + 1])
            indexfree += 2

    return compact_disk


def clean_up_disk_whole_files(tupledisk):
    set_used = set()

    for file_index in range(len(tupledisk) - 1, 0, -1):
        file = tupledisk[file_index]
        if file[0] != "free" and file not in set_used:
            set_used.add(file) # to make sure they can only be found ones

            # look for free space big enough
            for i, space in enumerate(tupledisk):
                if i >= file_index: # big enough space needs to be before
                    break

                if space[0] == "free":
                    if space[1] == file[1]:
                        # same space is change places
                        tupledisk[i], tupledisk[file_index] = tupledisk[file_index], tupledisk[i]
                        break
                    elif space[1] > file[1]:
                        # change part of free with disk
                        remaining = space[1] - file[1]
                        tupledisk[i], tupledisk[file_index] = file, ("free", file[1])
                        tupledisk.insert(i + 1, ("free", remaining))
                        break

    return tupledisk


def check_sum(compact_disk):
    checksum, position = 0, 0

    for block in compact_disk:
        file_id, length = block
        if file_id != "free":  # Only consider file blocks
            for _ in range(length):
                checksum += position * file_id
                position += 1
        else:  # Skip free space
            position += length

    return checksum


with open('/content/drive/MyDrive/Personal Project/Advent of Code/2024/Day_9/input_09_12_2024.txt') as f:
    lines = f.read().strip()
    tuple_space = [(filenumber if i % 2 == 0 else "free", int(line))
                   for i, line in enumerate(lines)
                   for filenumber in [i // 2]]

    print("Part 1, Checksum of compact disk:", check_sum(clean_up_disk(tuple_space)))
    print("Part 2, Checksum of ordered disk:", check_sum(clean_up_disk_whole_files(tuple_space)))

Part 1, Checksum of compact disk: 6471961544878
Part 2, Checksum of ordered disk: 6511178035564
